In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (10201 Collins Ave Unit 2301, Bal Harbour, FL ...
2     (100 South Pointe Dr APT 3901, Miami Beach, FL...
3     (1000 Biscayne Blvd #3001, Miami, FL 33132, US...
4     (5959 Collins Ave APT 1605, Miami Beach, FL 33...
5     (15701 Collins Ave #802, North Miami Beach, FL...
6     (16901 Collins Ave #3202, Sunny Isles Beach, F...
7     (17201 Biscayne Blvd #7, North Miami Beach, FL...
8     (16047 Collins Ave PH 3501, Sunny Isles Beach,...
9     (Santa Maria, 1643 Brickell Ave APT 2302, Miam...
10    (400 South Pointe Dr APT 2304, Miami Beach, FL...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Oceana Bal Harbour Condo\r10201 Collins Ave 23...,12/10/2019,1/20/2022,727,"$10,500,000.00",3992,"$2,630.26",Ivy Carneiro,Jennifer Brilliant,Oceana Bal Harbour Condo 10201 Collins Ave 230...,Oceana Bal Harbour Condo,10201 Collins Ave 2301 Bal Harbour,"(10201 Collins Ave Unit 2301, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
2,Continuum South Tower\r100 S Pointe Dr 3901\rM...,1/21/2022,1/21/2022,1,"$8,000,000.00",1940,"$4,123.71",Comp Sale,Jorge Martinez,Continuum South Tower 100 S Pointe Dr 3901 Mia...,Continuum South Tower,100 S Pointe Dr 3901 Miami Beach,"(100 South Pointe Dr APT 3901, Miami Beach, FL...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
3,One Thousand Museum\r1000 Biscayne Blvd 3001\r...,9/22/2021,1/20/2022,110,"$7,500,000.00",4600,"$1,630.43",Patrice Hallot,Anna Karan PA,One Thousand Museum 1000 Biscayne Blvd 3001 Miami,One Thousand Museum,1000 Biscayne Blvd 3001 Miami,"(1000 Biscayne Blvd #3001, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
4,The Bath Club\r5959 Collins Ave 1605\rMiami Beach,9/20/2021,1/21/2022,112,"$5,550,000.00",3807,"$1,457.84",Julian Johnston,Talita Pinheiro,The Bath Club 5959 Collins Ave 1605 Miami Beach,The Bath Club,5959 Collins Ave 1605 Miami Beach,"(5959 Collins Ave APT 1605, Miami Beach, FL 33...","(25.8424476, -80.1203048, 0.0)",25.842448,-80.120305,0.0
5,Ritz\r15701 Collins Ave 802\rSunny Isles Beach,9/24/2021,1/18/2022,54,"$3,625,000.00",2800,"$1,464.65",Alice Edery,Adina Bistritzky,Ritz 15701 Collins Ave 802 Sunny Isles Beach,Ritz,15701 Collins Ave 802 Sunny Isles Beach,"(15701 Collins Ave #802, North Miami Beach, FL...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0
6,Jade Signature Condo\r16901 Collins Ave 3202\r...,8/18/2021,1/20/2022,126,"$3,400,000.00",2494,"$1,363.27",Santiago Estevez Barroso,Pilar Mercader,Jade Signature Condo 16901 Collins Ave 3202 Su...,Jade Signature Condo,16901 Collins Ave 3202 Sunny Isles Beach,"(16901 Collins Ave #3202, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
7,Marina Palms\r17201 Biscayne Blvd PH-7\rAventura,8/14/2018,1/19/2022,1128,"$3,175,000.00",3646,$870.82,Rodolfo Maya,Rodolfo Maya,Marina Palms 17201 Biscayne Blvd PH-7 Aventura,Marina Palms,17201 Biscayne Blvd PH-7 Aventura,"(17201 Biscayne Blvd #7, North Miami Beach, FL...","(25.9343268, -80.1506647, 0.0)",25.934327,-80.150665,0.0
8,Turnberry Ocean Colony\r16047 Collins Ave 3501...,12/3/2021,1/18/2022,2,"$3,050,000.00",2772,"$1,100.29",Karen Matluck PA,Lana Bell,Turnberry Ocean Colony 16047 Collins Ave 3501 ...,Turnberry Ocean Colony,16047 Collins Ave 3501 Sunny Isles Beach,"(16047 Collins Ave PH 3501, Sunny Isles Beach,...","(25.9252659, -80.1216882, 0.0)",25.925266,-80.121688,0.0
9,Santa Maria Condo\r1643 Brickell Ave 2302\rMiami,4/30/2021,1/18/2022,220,"$2,950,000.00",3170,$930.60,Claudio Iglesias,Frederique Leforestier,Santa Maria Condo 1643 Brickell Ave 2302 Miami,Santa Maria Condo,1643 Brickell Ave 2302 Miami,"(Santa Maria, 1643 Brickell Ave APT 2302, Miam...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
10,South Pointe Towers Condo\r400 S Pointe Dr 230...,10/21/2021,1/19/2022,76,"$2,450,000.00",1802,"$1,359.60",Michael Feuling,Michael Feuling,South Pointe Towers Condo 400 S Pointe Dr 2304...,South Pointe Towers Condo,400 S Pointe Dr 2304 Miami Beach,"(400 South Pointe Dr APT 2304, Miami Beach, FL...","(25.7672546, -80.1342458, 0.0)",25.767255,-80.134246,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 16th - January 22nd')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [12]:
m.save('index.html')